# Jupyter Notebook Gruppe Tesla/Musk
Teammitglieder: Falk Jaresch, Michael Foeger, Paula Engelberg, Christoph Fink
    

## Import der Datenquelle

In [33]:
from rdflib import Graph
from pandas import DataFrame

# Serialisierung: Parsen von Turtle zu ntriple

In [34]:
g = Graph()
g.parse("tesla.ttl", format="ttl")
g.serialize(destination='tesla.nt', format="ntriples")

In [35]:
g.parse("tweets.ttl", format="ttl")
g.serialize(destination='tweets.nt', format='ntriples')

## Query

In [36]:
result = g.query("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>       
PREFIX dbo:  <http://dbpedia.org/ontology/>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX sioc:  <http://rdfs.org/sioc/ns#>
PREFIX dc:  <http://purl.org/dc/elements/1.1/>

SELECT ?date ?close

WHERE {
  ?date ex:close ?close .
}

ORDER BY DESC(?date)
""")

DataFrame(result, columns=result.vars)

,date,close
0,http://example.org/resource/elonmusk/tesla/9/9...,194.470001
1,http://example.org/resource/elonmusk/tesla/9/9...,248.910004
2,http://example.org/resource/elonmusk/tesla/9/9...,278.480011
3,http://example.org/resource/elonmusk/tesla/9/9...,160.699997
4,http://example.org/resource/elonmusk/tesla/9/9...,22.969999
...,...,...
1687,http://example.org/resource/elonmusk/tesla/1/1...,229.869995
1688,http://example.org/resource/elonmusk/tesla/1/1...,145.720001
1689,http://example.org/resource/elonmusk/tesla/1/1...,33.529999
1690,http://example.org/resource/elonmusk/tesla/1/1...,27.620001
